## _*H2 energy plot computed using ExcitationPreserving*_

This notebook demonstrates using Qiskit Nature to plot graphs of the ground state energy of the Hydrogen (H2) molecule over a range of inter-atomic distances using VQE and ExcitationPreserving. It is compared to the same energies as computed by the NumPyMinimumEigensolver. `ExcitationPreserving` is a particle preserving variational form and should be used in conjunction with operator `jordan_wigner mapping` and `HarteeFock` initial state.

In [5]:
import numpy as np
import pylab

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.formats import MoleculeInfo
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

from qiskit_algorithms import NumPyMinimumEigensolver

from qiskit_algorithms.optimizers import SLSQP

from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_algorithms import VQE

from qiskit.circuit.library import ExcitationPreserving
from qiskit.primitives import Estimator

In [29]:
distances  = [0.5 + 0.025*x  for x in range(0, 20)]

def compute_energies(algorithm, distances):
    print(f'{algorithm}, processing step __', end='')

    energies = []
    hf_energies = []
    estimator = Estimator()

    for i, d in enumerate(distances):
        print('\b\b{:2d}'.format(i), end='', flush=True)

        info = MoleculeInfo(["H", "H"], [(0.0, 0.0, 0.0), (0.0, 0.0, d)])
        driver = PySCFDriver.from_molecule(info, basis="sto3g")
        molecule = driver.run()
        hamiltonian = molecule.hamiltonian.second_q_op()
        mapper = JordanWignerMapper()
        tapered_mapper = molecule.get_tapered_mapper(mapper)
        qubit_op = tapered_mapper.map(hamiltonian)

        if algorithm == 'NumPyMinimumEigensolver':
            algo = NumPyMinimumEigensolver()
            algo.filter_criterion = molecule.get_default_filter_criterion()
            solver = GroundStateEigensolver(mapper, algo)
            result = solver.solve(molecule)
            hf_energies.append(result.hartree_fock_energy)
            energies.append(result.total_energies[0])

        elif algorithm == 'VQE':
            optimizer = SLSQP(maxiter=10000, ftol=1e-9)
            ansatz = ExcitationPreserving(4, reps=5) # https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.library.ExcitationPreserving for all parameters
            vqe = VQE(estimator, ansatz, optimizer)
            algo = GroundStateEigensolver(tapered_mapper, vqe)
            result = algo.solve(molecule)
            energies.append(result.total_energies[0])
            ansatz.decompose().draw()

    print(' --- complete')
    return hf_energies, energies

In [30]:
hf_energies, npme_energies = compute_energies('NumPyMinimumEigensolver', distances)
_, vqe_energies = compute_energies('VQE', distances)

NumPyMinimumEigensolver, processing step 19 --- complete
VQE, processing step 19 --- complete


In [32]:
! pip freeze | grep qiskit

qiskit==1.1.0
qiskit-aer==0.14.2
qiskit-algorithms==0.3.0
qiskit-ibm-runtime==0.25.0
qiskit-machine-learning==0.7.2
qiskit-nature==0.7.2
qiskit-nature-pyscf==0.4.0
qiskit-qasm3-import==0.5.0
qiskit-transpiler-service==0.4.5
